In [3]:
import sys
import pandas as pd 
import numpy as np
#np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import pathlib
%matplotlib inline

In [7]:
%%time
data_dir = '../data/raw/Yaw-01/'
data_path = pathlib.Path(data_dir)

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 18.1 µs


In [14]:
for f in data_path.glob("*int"):  
    print(f)
    

../data/raw/Yaw-01/0001.int
../data/raw/Yaw-01/0002.int
../data/raw/Yaw-01/0003.int
../data/raw/Yaw-01/0004.int
../data/raw/Yaw-01/0005.int


In [21]:
sensor = (1,4,5,6,7,8,9,10,11,12,53,54,56,57,59,60,61,62,63,64)


def read_int(FileName, x):

    #print(FileName)

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending=='.int':
        FileName = FileName[0:xlen-4]

    with open(FileName+ending,"rb") as f:
        f.read()
        filesize = f.tell()
    f.close()

    with open(FileName+ending,"rb") as f:

    # the first 4 junk buffer

        leer = np.frombuffer(f.read(4),dtype=np.single)
        print(leer)
    # the  24 buffer give information about file
        datei = np.frombuffer(f.read(24),dtype=np.uint32)
        print(datei)

        f.seek(76)
    # read the number of sensor in file 
        lvek = np.frombuffer(f.read(4),dtype=np.uint32)
        print(lvek)
     
    # then 4 times the number of sensors are skipped to read a control parameter:
  
        f.seek(84+lvek[0]*4)

        filetype = np.frombuffer(f.read(4),dtype=np.uint)
        print(filetype)

        if filetype != 12:

    # start time is read (is not equal to 0 as we have been cut off from the simulation for the first 50 s
            t0 = np.frombuffer(f.read(4),dtype=np.single)
            #print(t0)

    # comes the parameter for the step size of the simulation

            dt = np.frombuffer(f.read(4),dtype=np.single)
            #print(dt)

            pos_fak = f.tell()
            #print(pos_fak)

            fak = []
 
            for n in x:
                f.seek(pos_fak + 4*(n-1))
                fak.append(np.frombuffer(f.read(4),dtype=np.single))
                
            f.seek(pos_fak+lvek[0]*4)
            position = f.tell()
    # the length of the sensors is calculated:
            RecordCount = round((filesize-position)/lvek[0]/2)
            #print(RecordCount)

            raw_ts_int = np.empty([len(x),RecordCount])
           # print(raw_ts_int)
        
            #print("Type:",type(raw_ts_int))
            #print(raw_ts_int.shape)

            for n in range(RecordCount):

                for m, sensor_ in enumerate(x):

                    f.seek(position+2*sensor_+(lvek[0]*2)*n-2)
                    raw_ts_int[m,n] = np.frombuffer(f.read(2),dtype=np.int16)*fak[m]

            global   df
            df = pd.DataFrame(raw_ts_int)

            #print(df.transpose())
            #merged_df=df.merge(df)
                    
    f.close()

In [22]:
read_int(str(data_path / '0001.int'),sensor)

[1.01e-43]
[   8   14 2022   15   27    1]
[65]


ValueError: buffer size must be a multiple of element size